In [1]:
from autogen import ConversableAgent
from dotenv import load_dotenv
import os
from openai import OpenAI
from pathlib import Path

In [56]:
# 加载 .env 文件
load_dotenv()

True

## Kimi API

In [3]:
kimi_weak_llm_config = {
    "config_list": [
        {
            "model": "moonshot-v1-8k",
            "api_key": os.environ.get("Kimi_API_KEY"),
            "api_type": "openai",
            "base_url": "https://api.moonshot.cn/v1",
            "price": [0.012,0.012]
        }],
    "temperature": 0.1,
    "timeout": 300
}

kimi_strong_llm_cofig = {
    "config_list": [
        {
            "model": "moonshot-v1-128k",
            "api_key": os.environ.get("Kimi_API_KEY"),
            "api_type": "openai",
            "base_url": "https://api.moonshot.cn/v1",
            "price": [0.06,0.06]
        }],
    "temperature": 0.1,
    "timeout": 300
}

## Ali api

In [65]:
Qwen_weak_llm_config = {
    "config_list": [
        {
            "model": "qwen-plus",
            "api_key": os.environ.get("Qwen_API_KEY"),
            "api_type": "openai",
            "base_url": "https://dashscope.aliyuncs.com/compatible-mode/v1",
            "price": [0.004,0.012]
        }],
    "temperature": 0.5,
    "timeout": 300
}

Qwen_strong_llm_config = {
    "config_list": [
        {
            "model": "qwen-max",
            "api_key": os.environ.get("Qwen_API_KEY"),
            "api_type": "openai",
            "base_url": "https://dashscope.aliyuncs.com/compatible-mode/v1",
            "price": [0.004,0.012]
        }],
    "temperature": 0.5,
    "timeout": 300
}

In [66]:
os.environ.get("Qwen_API_KEY")

'sk-9e1015602c4b48b3abd216a5feb50687'

In [5]:
# Kimi的文件接口
client = OpenAI(
    api_key = os.environ.get("Kimi_API_KEY"),
    base_url = "https://api.moonshot.cn/v1",
)

In [67]:
agent = ConversableAgent(
    "chatbot",
    llm_config=Qwen_weak_llm_config,
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [70]:
reply = agent.generate_reply(messages=[{"content": "给我讲个笑话.", "role": "user"}])

In [71]:
print(reply)

当然可以！接下来是一个简短又有趣的笑话：

为什么袜子总是只丢一只？因为丢两只根本就不会发现。


## 文件提取

In [15]:
# 使用kimi api读取
def file_extraction(file_path):
    file_object = client.files.create(file=Path(file_path), purpose="file-extract")
    file_content = client.files.content(file_id=file_object.id).text
    return file_content

In [16]:
text = file_extraction("test files/象上-文稿-转写结果.docx")

In [26]:
PPT = file_extraction("test files/上海象上投资.pptx")

## Agent设计

In [ ]:
def report_writing()

In [17]:
extraction_assistant = ConversableAgent(
    name = "信息提取员",
    system_message = "你是一个信息提取员，任务是将我需要的信息，从我给你的一大段文字中提取出来，并返回给我",
    llm_config = llm_config
)

In [47]:
requirements = "配置逻辑 1这个部分是尽调报告的关键, 以下是量化策略的模板, 量化策略较为标准化, ⽐较⽅便进⾏模块化的抽取, 以下内容都需要进⾏抽取6%- 管理⼈量化策略的整体思路｡ 例如% 整体量化思路以强逻辑为主, 对因⼦的可解释性有很⾼的要求- 管理⼈的因⼦组成和数量, 例如% 共200个，量价类 30%, 基本⾯类 40%, 另类因⼦ 30%- 管理⼈对因⼦的解释定义, 例如% 另类因⼦主要包含公司公告, 新闻舆情, 专利信息等- 管理⼈的因⼦挖掘⽅法, 例如% ⼤部分为⼿⼯挖掘, 较少/不使⽤机器学习- 管理⼈的因⼦处理⽅法, 例如% 会使⽤机器学习将因⼦进⾏组合/会根据相关性把因⼦分组等/⾏业中性/市值中性等- 管理⼈的量化模型% 例如% 以线性模型为主, 具体为排序打分, XXX- 管理⼈的因⼦⼊库条件, 例如% 会在模拟盘中观察 X ⽉, 因⼦相关性不⾼于XX, 关注 ICIR/夏普, 等- 管理⼈的具体执⾏, 例如% 模型对股票进⾏ XXX 处理, 最后选出 XXX 个个股, 使⽤⾃研交易算法进⾏下单等- 管理⼈的对冲⽅法 1如有6% 使⽤什么进⾏对冲, 有⽆敞⼝4. 持仓特征% 管理⼈持有多少只个股, 指数内占⽐等5. 交易⻛格% 年化双边换⼿率 XX 倍 1数据抽取66. 容量% 管理⼈的策略能承受多⼤的资⾦, 如% XX 亿 1数据抽取6"

In [48]:
integrate_assistant = ConversableAgent(
    name="报告撰写员",
    system_message=f"你是一个报告撰写员，需要根据我发给你的字段，根据我的要求，详细的写出一份报告，并返回给我,以下是要求：不需要面面俱到，写出提供材料中包含的即可：{requirements}",
    llm_config = strong_llm_cofig
)

In [20]:
quant_idea = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "管理⼈量化策略的整体思路, 例如，是偏强逻辑为主, 对因⼦的可解释性有很⾼的要求；还是偏暴力挖掘为主，使用机器学习较多", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [21]:
print(quant_idea)

在管理人量化策略的整体思路方面，对话中提到了以下几点：

1. **因子挖掘**：大部分因子是手工挖掘出来的，有一位同事正在做机器学习的部分（说话人3 05:47）。
2. **因子权重**：因子的权重会根据历史表现和因子之间的相关性进行调整（说话人3 08:10）。
3. **机器学习应用**：使用机器学习主要是为了预测不同策略的表现，并据此调整策略权重（说话人3 08:44）。
4. **策略类型**：提到了子策略的概念，这些子策略可能使用不同的因子和处理方式，如量价因子、基本面因子等（说话人3 09:23）。
5. **因子逻辑**：因子都是有逻辑的因子，强调了因子的可解释性（说话人1 05:58）。

综上所述，管理人的量化策略整体思路既注重因子的逻辑性和可解释性，也利用机器学习技术进行策略优化和预测。他们并没有完全依赖于暴力挖掘，而是在保持因子逻辑性的基础上，适当引入机器学习技术来提升策略的表现。


In [30]:
factor_composition = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "从文件中提取出管理人的策略中的因子组成和因子数量", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [31]:
print(factor_composition)

管理人的策略主要由以下三类因子组成：基本面因子占40%，另类因子占10%，量价因子占50%。这些权重占比指的是因子数量上的权重，而非对收益的贡献占比。目前，管理人的因子库中大概有三五十个因子，实盘中使用的因子数量大约为30个。这些因子主要是通过手工挖掘出来的，但有一位同事正在研究机器学习的部分。


In [34]:
factor_explanation = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "从文件中提取出管理人对其不同因子类型的解释（如有），具体来说主要是不同因子是基于哪些数据锝出的，不要编造，只能抽取提供信息中有的内容。如果没有详细说明，则返回无", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [29]:
print(factor_explanation)

在提供的文本中，管理人对其不同因子类型的解释主要涉及以下几个方面：

1. 因子的构成：管理人提到他们的量化模型主要基于三类因子，分别是基本面因子（占比40%）、另类因子（占比10%）和量价因子（占比50%）。

2. 因子的挖掘：管理人明确表示，他们的因子主要是手工挖掘出来的，并且有一位同事正在研究机器学习的部分。

3. 因子的逻辑性：管理人强调他们的因子都是有逻辑的，即每个因子都有其背后的逻辑支持。

4. 因子的测试和入库：管理人提到，他们会对因子进行单因子测试，只有通过一定检验的因子，如多空收益和夏普比率等，才会被放入数据库。

5. 因子的组合和权重调整：管理人说明他们会根据因子的历史表现和因子之间的相关性来调整不同因子的权重。

6. 因子的更新和淘汰：管理人表示他们会持续跟踪因子的表现，并根据市场变化对因子的权重进行调整，如果某个因子表现不佳，可能会被暂时搁置，但不会完全放弃，因为不同市场风格下，某些因子可能会重新变得有效。

7. 数据来源：管理人提到他们使用的量化数据主要来源于通联数据，而不是同花顺，因为他们认为同花顺的数据对于他们的策略增强效果一般。

8. 舆情系统：虽然提到了舆情系统，但管理人说明这个系统目前还没有用到实盘策略中，主要用于抓取不同平台的信息，可能在盘后的消息中反映在调仓决策里。

以上是文本中提到的关于不同因子类型的相关信息，没有编造或添加额外内容。


In [36]:
model_explanation = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "从文件中提取出管理人的建模方法，需要明确提示是线性还是机器学习模型或者是两者都有及其偏好，详述他们的建模方法论, 例如% 以线性模型为主, 具体为排序打分, XXX", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [37]:
print(model_explanation)

根据文件内容，管理人的建模方法主要包括以下几个方面：

1. **模型类型**：管理人使用的是线性模型，同时也有机器学习模型的参与。他们主要基于基本面、另类和量价三类因子构建量化模型。

2. **因子权重**：模型中，基本面因子占40%，另类因子占10%，量价因子占50%。这些权重指的是因子数量上的占比，而非对收益的贡献占比。

3. **因子挖掘**：大部分因子是手工挖掘出来的，但有一位同事正在研究机器学习的部分。

4. **因子测试与选择**：在单因子测试阶段，会检验因子的多空收益、夏普比率等指标，满足一定要求后才会加入数据库。然后结合其他因子，考虑其相关性和对组合的贡献度，决定是否加入实盘策略。

5. **因子调整**：因子的权重会基于历史表现和因子间的相关性进行调整。如果因子历史表现好，会给予更高的权重；如果两个因子相关性很高，相关性高的第二个因子的权重会降低。

6. **机器学习应用**：使用机器学习主要是为了预测不同子策略的表现，并据此调整策略权重。这些子策略可能使用不同的因子集合，形成不同的打分模型。

7. **模型频率**：在量价信号上，最高使用到一分钟的频率，但调仓操作以日频为主，将高频信号转换成低频信号。

8. **模型策略**：模型包括多头策略和市场中性策略，其中市场中性策略是在多头策略的基础上加入对冲端。

9. **风险控制**：会跟踪行业偏离度和风格因子的偏离，并进行一定限制。持仓数量大约在200-300只，个股权重最大不超过2%。

10. **模型迭代**：模型的迭代每天都在进行，新因子的加入会不断完善模型。

11. **舆情系统**：虽然开发了舆情系统，但目前尚未应用到实盘策略中。

12. **T0增强**：使用第三方的T0增强策略。

13. **换手率**：换手率较高，大约在60-80倍，但不包括T0交易部分。

14. **因子贡献**：最近三年数据显示，盈利、价值和股息率因子对收益的贡献较大。

15. **模型运行**：指针策略从2023年开始运行，之前有其他策略，后来调整为只做指针策略。

16. **资金容量**：目前没有考虑资金容量，但估计10-20亿的资金规模不会对收益产生衰减。

17. **规模和资金来源**：目前总体规模为两个多亿，其中70%为自有资金。

通过上述信息，可以看出管理人的建模方

In [38]:
factor_update = extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "从文件中提取出管理人的新因子入库条件，例如：会在模拟盘中观察 X ⽉, 因⼦相关性不⾼于XX, 关注 ICIR/夏普", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [39]:
print(factor_update)

根据文件内容，管理人的新因子入库条件主要包括以下几点：

1. 因子需要通过单因子测试，满足一定的检验标准，如多空收益、SL等数值达到要求。
2. 因子会放入数据库中，结合其他因子进行相关性分析，看其对组合的贡献是否明显。
3. 如果因子表现不佳，可能会作为储备因子放入因子库中。
4. 因子的权重会基于历史表现和因子之间的相关性进行调整。
5. 管理人会使用机器学习预测不同子策略的表现，并据此调整权重。
6. 因子的类型会影响其在模拟盘中的观察时间，高频因子可能观察两三个月，低频因子如基本面可能观察半年左右。
7. 管理人会定期跟踪因子的表现，并根据历史表现和市场风格的变化进行权重调整。

文件中没有明确提到“ICIR/夏普”作为新因子入库的条件。


In [40]:
strategy_excecution= extraction_assistant.generate_reply(messages=[{"content": text, "role":"system"},
                                                      #{"content": "从文件中提取出有关管理人组织架构相关的信息，包括公司一共有多少人以及他们的分工.", "role": "user"},
                                                      {"content": "从文件中提取出管理人的具体操作，例如有无使用自研/外部的下单算法，不同策略的持股数是多少，年化双边换手率多少倍等", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [41]:
print(strategy_excecution)

以下是从文件中提取的管理人的具体操作信息：

1. **公司成立与背景**：
   - 成立于2021年，最初注册在商登网，后转到上海。
   - 创始人曾在同花顺工作，对交易感兴趣，离职后专注于股票量化交易。

2. **团队构成**：
   - 策略研究员4人。
   - 后台开发管理3人，负责风控管理后台。
   - 交易员2人。

3. **主要策略**：
   - 量化中性策略。
   - 指数增强策略。

4. **产品管理**：
   - 杨总作为基金经理，管理所有产品。

5. **因子构成**：
   - 40%基本面因子。
   - 10%另类因子。
   - 50%量价因子。

6. **因子挖掘**：
   - 主要手工挖掘，有一位同事正在研究机器学习。

7. **因子库规模**：
   - 具体个数未统计，但至少有三五十个因子。

8. **因子测试与组合**：
   - 单因子测试，通过多空收益、SL等指标筛选。
   - 结合其他因子，考虑相关性和组合贡献。

9. **因子权重调整**：
   - 基于历史表现和因子间相关性调整权重。

10. **机器学习应用**：
    - 用于预测不同策略的表现，调整策略权重。

11. **策略频率**：
    - 最高使用一分钟频率的量价信号，但调仓以日频为主。

12. **对冲工具**：
    - 主要使用IC（股指期货）进行对冲。

13. **因子入库与出库**：
    - 持续跟踪因子表现，与历史表现对比，必要时调整权重或出库。

14. **新因子测试**：
    - 高频因子模拟两三个月，低频因子可能模拟半年。

15. **数据来源**：
    - 不使用同花顺数据，主要使用通联数据。

16. **舆情系统**：
    - 自行开发，抓取多平台信息，但暂未用于实盘策略。

17. **持仓数量**：
    - 单个产品持仓约500只股票。

18. **个股权重限制**：
    - 最大不超过2%。

19. **行业偏离度**：
    - 限制在5%~10%。

20. **风格因子控制**：
    - 控制在相对于基准的15%以内。

21. **市值暴露**：
    - 限制在30%以内。

22. **持仓市值中位数**：
    - 大约

In [43]:
components_list = [quant_idea,factor_composition,factor_explanation,model_explanation,factor_update,strategy_excecution]

In [53]:
final_report= integrate_assistant.generate_reply(messages=[{"content": str(components_list), "role":"system"},
                                                      {"content": "按需求生成报告，不要分段，写成一整段，要求逻辑清晰，细节完整，但不要有过多的你自由发挥的解释，客观记录为主", "role": "user"}])


>>>>>>>> USING AUTO REPLY...


In [55]:
print(final_report)

根据提供的资料，管理人的量化策略整体思路强调因子的逻辑性和可解释性，同时适当引入机器学习技术进行策略优化和预测。其策略主要由基本面因子（占比40%）、另类因子（占比10%）、量价因子（占比50%）组成，这些权重占比指的是因子数量上的权重而非对收益的贡献占比。管理人的因子库中大约有三五十个因子，实盘中使用的因子数量大约为30个，这些因子主要是通过手工挖掘出来的，但有一位同事正在研究机器学习的部分。

在因子的挖掘和测试方面，管理人强调因子都是有逻辑的，即每个因子都有其背后的逻辑支持。他们会对因子进行单因子测试，只有通过一定检验的因子，如多空收益和夏普比率等，才会被放入数据库。因子的权重会根据因子的历史表现和因子之间的相关性进行调整，如果因子历史表现好，会给予更高的权重；如果两个因子相关性很高，相关性高的第二个因子的权重会降低。

管理人使用的是线性模型，同时也有机器学习模型的参与。他们主要基于基本面、另类和量价三类因子构建量化模型。使用机器学习主要是为了预测不同子策略的表现，并据此调整策略权重。这些子策略可能使用不同的因子集合，形成不同的打分模型。

在风险控制方面，管理人会跟踪行业偏离度和风格因子的偏离，并进行一定限制。持仓数量大约在200-300只，个股权重最大不超过2%。模型的迭代每天都在进行，新因子的加入会不断完善模型。虽然开发了舆情系统，但目前尚未应用到实盘策略中。

换手率方面，管理人的策略换手率较高，大约在60-80倍，但不包括T0交易部分。最近三年数据显示，盈利、价值和股息率因子对收益的贡献较大。资金容量方面，目前没有考虑资金容量，但估计10-20亿的资金规模不会对收益产生衰减。目前总体规模为两个多亿，其中70%为自有资金。

管理人的新因子入库条件包括通过单因子测试满足一定检验标准，放入数据库中进行相关性分析，观察因子表现并根据历史表现和市场风格的变化进行权重调整。高频因子可能观察两三个月，低频因子如基本面可能观察半年左右。管理人会定期跟踪因子的表现，并根据市场变化对因子的权重进行调整。
